In [116]:
import pylibxc
import numpy as np
import pandas as pd

file = 'gga_c'
df = pd.read_csv(f'{file}.csv', na_values='---')

xc_col = file.replace('_c','').upper() + ' corr.'
df.rename(columns={'xc':xc_col}, inplace=True)


In [117]:
bib_file = []
ref_names = []
for xc in df[xc_col]:
  func_c = pylibxc.LibXCFunctional(xc, "polarized")

  ref_name='\cite{'
  for ref in func_c.get_bibtex():
    bib_file.append(ref)
    ref_name += ref.splitlines()[0].replace('@article{','').replace('@inproceedings{','')
  ref_name = ref_name[:-1] + '}' 

  ref_names.append(ref_name)

df['Refs.'] = ref_names 

# write out bib file
bib_file = "\n".join(bib_file)
f = open(f"{file}.bib", "w")
f.write(bib_file)
f.close

<function TextIOWrapper.close()>

In [118]:
# format functional names
df[xc_col] = df[xc_col].apply(lambda row: row.replace('gga_c_','').replace('_','\_'))

# format columns
for c in df.columns:
  if 'range' in c:
    df[c] = df[c].apply(lambda row: np.NaN if row is np.NaN else np.array(row[1:-1].split(', '), dtype=np.float64))
    df[c] = df[c].apply(lambda row: np.around(row, decimals=4))

df = df.fillna('---')

In [119]:
with pd.option_context("max_colwidth", 1000):
    latex = df.to_latex(index=False, escape=False, column_format='|c|c|c|c|c|l|')

# horizontal line sep
latex = latex.replace("\\\n", "\\ \hline\n").replace('\\toprule','\\hline')

latex_list = latex.splitlines()
latex=[]
for line in latex_list:
  if '\\toprule' in line or '\\midrule' in line or '\\bottomrule' in line:
    continue
  else:
    latex.append(line)



latex = "\n".join(latex)

f = open(f"{file}.tex", "w")
f.write(latex)
f.close

print(latex)

\begin{tabular}{|c|c|c|c|c|l|}
\hline
          GGA C &  satisfied condition? &       $r_s$ range &         $s$ range &    $\zeta$ range  &                                                           Refs. \\ \hline
          acgga &                  True &               --- &               --- &               --- &                         \cite{Cancio2018_084116,Burke2014_4834} \\ \hline
         acggap &                  True &               --- &               --- &               --- &                         \cite{Cancio2018_084116,Burke2014_4834} \\ \hline
           am05 &                  True &               --- &               --- &               --- &                  \cite{Armiento2005_085108,Mattsson2008_084714} \\ \hline
           apbe &                  True &               --- &               --- &               --- &                                    \cite{Constantin2011_186406} \\ \hline
            bmk &                 False &  [0.0001, 4.9975] &        [0.0, 5.0] & 

In [120]:
# save updated 
df.to_csv('gga_c_final.csv', index=None)